# 제품 이상여부 판별 프로젝트

# 문제 정의
1. 기포 발생
    - 탈포 과정에서 기포를 제거하는데 압력과 시간에 따라 기포가 남아있을 수 있음
    - 탈포 과정: 진공 챔버에서 미세 기포를 제거하는 과정
2. 디스플레이 틀어짐
    - 모름
3. 글라스 깨짐
    - 합착이나 경화 과정에서 레진의 수축으로 인하여 일어남
    - 합착 과정: 글라스를 자유낙하하여 자중으로 붙이는 과정
    - 완정 경화 과정: 전체적으로 UV 조사하여 디스플레이와 글라스의 합착력을 최대화 하는 과정

## 1. 데이터 불러오기

### 필수 라이브러리

In [ ]:
!pip install catboost

In [ ]:
!pip install shap

In [1]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

pd.set_option('display.max_columns', None)

## Seed 고정

In [2]:
import random

# Seed 고정
SEED = 42

# numpy seed 고정
np.random.seed(SEED)
# python random seed 고정
random.seed(SEED)
# os 환경 변수에서 seed 고정
os.environ['PYTHONHASHSEED'] = str(SEED)

### csv 파일 읽어오기

In [3]:
ROOT_DIR = "data"

# Load data
train = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
test = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))
train

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Z Unit Time_Dam,CURE END POSITION Z Judge Value_Dam,CURE END POSITION Θ Collect Result_Dam,CURE END POSITION Θ Unit Time_Dam,CURE END POSITION Θ Judge Value_Dam,CURE SPEED Collect Result_Dam,CURE SPEED Unit Time_Dam,CURE SPEED Judge Value_Dam,CURE STANDBY POSITION X Collect Result_Dam,CURE STANDBY POSITION X Unit Time_Dam,CURE STANDBY POSITION X Judge Value_Dam,CURE STANDBY POSITION Z Collect Result_Dam,CURE STANDBY POSITION Z Unit Time_Dam,CURE STANDBY POSITION Z Judge Value_Dam,CURE STANDBY POSITION Θ Collect Result_Dam,CURE STANDBY POSITION Θ Unit Time_Dam,CURE STANDBY POSITION Θ Judge Value_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION X Unit Time_Dam,CURE START POSITION X Judge Value_Dam,CURE START POSITION Z Collect Result_Dam,CURE START POSITION Z Unit Time_Dam,CURE START POSITION Z Judge Value_Dam,CURE START POSITION Θ Collect Result_Dam,CURE START POSITION Θ Unit Time_Dam,CURE START POSITION Θ Judge Value_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED SPEED OF RESIN Unit Time_Dam,DISCHARGED SPEED OF RESIN Judge Value_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Unit Time_Dam,DISCHARGED TIME OF RESIN(Stage1) Judge Value_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Unit Time_Dam,DISCHARGED TIME OF RESIN(Stage2) Judge Value_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Unit Time_Dam,DISCHARGED TIME OF RESIN(Stage3) Judge Value_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage1) Unit Time_Dam,Dispense Volume(Stage1) Judge Value_Dam,Dispense Volume(Stage2) Collect Result_Dam,Dispense Volume(Stage2) Unit Time_Dam,Dispense Volume(Stage2) Judge Value_Dam,Dispense Volume(Stage3) Collect Result_Dam,Dispense Volume(Stage3) Unit Time_Dam,Dispense Volume(Stage3) Judge Value_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Unit Time_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Unit Time_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Judge Value_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Unit Time_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Judge Value_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage1) Unit Time_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage1) Judge Value_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage2) Unit Time_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage2) Judge Value_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage3) Unit Time_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage3) Judge Value_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage1) Unit Time_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage1) Judge Value_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage2) Unit Time_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage2) Judge Value_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage3) Unit Time_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage3) Judge Value_Dam,HEAD Standby Position X Collect Result_Dam,HEAD Standby Position X Unit Time_Dam,HEAD Standby Position X Judge Value_Dam,HEAD Standby Position Y Collect Result_Dam,HEAD Standby Position Y Unit Time_Dam,HEAD Standby Position Y Judge Value_Dam,HEAD Standby Position Z Collect Result_Dam,HEAD Standby Position Z Unit Time_Dam,HEAD Standby Position Z Jud

## 필요없는 column 정리

In [4]:
trn_cols = train.columns[:-1].tolist()

# 1. Dam
dam_cols = [col for col in trn_cols if "_Dam" in col]

# 1) 컬럼 중 전체가 결측치인 컬럼 제외
dam_main_cols = [col for col in dam_cols if train[col].nunique() >= 1]

trn_dam = train[dam_main_cols]

# 2) 값이 1개인 컬럼 drop
drop_trn_dam = trn_dam.drop(columns=[
    "Wip Line_Dam", 
    "Process Desc._Dam",
    "Insp. Seq No._Dam",
    "Insp Judge Code_Dam",
    "CURE STANDBY POSITION X Collect Result_Dam",
    "CURE STANDBY POSITION Z Collect Result_Dam",
    "CURE STANDBY POSITION Θ Collect Result_Dam",
    "CURE START POSITION Z Collect Result_Dam",
])

# 3) 결측치, 이상치 대체, 값 변환
# drop_trn_dam["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam"] = drop_trn_dam["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam"].replace("OK", np.nan).astype(float)
# drop_trn_dam["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam"].fillna(drop_trn_dam["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam"].mean(), inplace=True)
# drop_trn_dam["HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam"] = drop_trn_dam["HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam"].fillna("NG").replace({"OK": 0, "NG": 1}).astype(int)


# 2. Fill1
fill1_cols = [col for col in trn_cols if "_Fill1" in col]

# 1) 컬럼 중 전체가 결측치인 컬럼 제외
fill1_main_cols = [col for col in fill1_cols if train[col].nunique() >= 1]

trn_fill1 = train[fill1_main_cols]

# 2) 값이 1개인 컬럼 drop
drop_trn_fill1 = trn_fill1.drop(columns=[
    "Wip Line_Fill1", 
    "Process Desc._Fill1",
    "Insp. Seq No._Fill1",
    "Insp Judge Code_Fill1",
])

# 3) 결측치, 이상치 대체, 값 변환
# drop_trn_fill1["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1"] = drop_trn_fill1["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1"].replace("OK", np.nan).astype(float)
# drop_trn_fill1["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1"].fillna(drop_trn_fill1["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1"].mean(), inplace=True)
# drop_trn_fill1["HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1"] = drop_trn_fill1["HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1"].fillna("NG").replace({"OK": 0, "NG": 1}).astype(int)


# 3. Fill2
fill2_cols = [col for col in trn_cols if "_Fill2" in col]

# 1) 컬럼 중 전체가 결측치인 컬럼 제외
fill2_main_cols = [col for col in fill2_cols if train[col].nunique() >= 1]

trn_fill2 = train[fill2_main_cols]

# 2) 값이 1개인 컬럼 drop
drop_trn_fill2 = trn_fill2.drop(columns=[
    "Wip Line_Fill2", 
    "Process Desc._Fill2",
    "Insp. Seq No._Fill2",
    "Insp Judge Code_Fill2",
    "CURE END POSITION Θ Collect Result_Fill2",
    "CURE STANDBY POSITION X Collect Result_Fill2",
    "CURE STANDBY POSITION Θ Collect Result_Fill2",
    "CURE START POSITION Θ Collect Result_Fill2",
    "DISCHARGED SPEED OF RESIN Collect Result_Fill2",
    "DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill2",
    "DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill2",
    "DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill2",
    "Dispense Volume(Stage1) Collect Result_Fill2",
    "Dispense Volume(Stage2) Collect Result_Fill2",
    "Dispense Volume(Stage3) Collect Result_Fill2"
])

# 3) 결측치, 이상치 대체, 값 변환
# drop_trn_fill2["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2"] = drop_trn_fill2["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2"].replace("OK", np.nan).astype(float)
# drop_trn_fill2["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2"].fillna(drop_trn_fill2["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2"].mean(), inplace=True)
# drop_trn_fill2["HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2"] = drop_trn_fill2["HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2"].fillna("NG").replace({"OK": 0, "NG": 1}).astype(int)


# 4. AutoClave
autoclave_cols = [col for col in trn_cols if "_AutoClave" in col]

# 1) 컬럼 중 전체가 결측치인 컬럼 제외
autoclave_main_cols = [col for col in autoclave_cols if train[col].nunique() >= 1]

trn_autoclave = train[autoclave_main_cols]

# 2) 값이 1개인 컬럼 drop
drop_trn_autoclave = trn_autoclave.drop(columns=[
    "Wip Line_AutoClave", 
    "Process Desc._AutoClave",
    "Equipment_AutoClave",
    "Insp. Seq No._AutoClave",
    "Insp Judge Code_AutoClave",
    "1st Pressure Judge Value_AutoClave",
    "2nd Pressure Judge Value_AutoClave",
    "3rd Pressure Judge Value_AutoClave",
])

# 3) 결측치, 이상치 대체, 값 변환
# drop_trn_autoclave["GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave"] = drop_trn_autoclave["GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave"].fillna("NG").replace({"OK": 0, "NG": 1}).astype(int)
# drop_trn_autoclave["GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave"] = drop_trn_autoclave["GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave"].fillna("NG").replace({"OK": 0, "NG": 1}).astype(int)
# drop_trn_autoclave["Chamber Temp. Judge Value_AutoClave"] = drop_trn_autoclave["Chamber Temp. Judge Value_AutoClave"].replace({"OK": 0, "NG": 1}).astype(int)


# 5. 데이터 병합
print(train.shape)
train_concat = pd.concat([drop_trn_dam, 
                          drop_trn_fill1, 
                          drop_trn_fill2, 
                          drop_trn_autoclave,
                          train["target"]], axis=1)

# 값 변환
train_concat["target"] = train_concat["target"].replace({"Normal": 0, "AbNormal": 1}).astype(int)

print(drop_trn_dam.shape[1],
      drop_trn_fill1.shape[1],
      drop_trn_fill2.shape[1],
      drop_trn_autoclave.shape[1], 
      train_concat.shape)

(40506, 468)
71 35 34 14 (40506, 155)


In [5]:
tst_cols = test.columns[1:-1].tolist()

# 1. Dam
dam_cols = [col for col in tst_cols if "_Dam" in col]

# 1) 컬럼 중 전체가 결측치인 컬럼 제외
dam_main_cols = [col for col in dam_cols if test[col].nunique() >= 1]

tst_dam = test[dam_main_cols]

# 2) 값이 1개인 컬럼 drop
drop_tst_dam = tst_dam.drop(columns=[
    "Wip Line_Dam", 
    "Process Desc._Dam",
    "Insp. Seq No._Dam",
    "Insp Judge Code_Dam",
    "CURE STANDBY POSITION X Collect Result_Dam",
    "CURE STANDBY POSITION Z Collect Result_Dam",
    "CURE STANDBY POSITION Θ Collect Result_Dam",
    "CURE START POSITION Z Collect Result_Dam",
])

# 3) 결측치, 이상치 대체, 값 변환
# drop_tst_dam["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam"] = drop_tst_dam["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam"].replace("OK", np.nan).astype(float)
# drop_tst_dam["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam"].fillna(drop_tst_dam["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam"].mean(), inplace=True)
# drop_tst_dam["HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam"] = drop_tst_dam["HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam"].fillna("NG").replace({"OK": 0, "NG": 1}).astype(int)


# 2. Fill1
fill1_cols = [col for col in tst_cols if "_Fill1" in col]

# 1) 컬럼 중 전체가 결측치인 컬럼 제외
fill1_main_cols = [col for col in fill1_cols if test[col].nunique() >= 1]

tst_fill1 = test[fill1_main_cols]

# 2) 값이 1개인 컬럼 drop
drop_tst_fill1 = tst_fill1.drop(columns=[
    "Wip Line_Fill1", 
    "Process Desc._Fill1",
    "Insp. Seq No._Fill1",
    "Insp Judge Code_Fill1",
])

# 3) 결측치, 이상치 대체, 값 변환
# drop_tst_fill1["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1"] = drop_tst_fill1["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1"].replace("OK", np.nan).astype(float)
# drop_tst_fill1["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1"].fillna(drop_tst_fill1["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1"].mean(), inplace=True)
# drop_tst_fill1["HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1"] = drop_tst_fill1["HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1"].fillna("NG").replace({"OK": 0, "NG": 1}).astype(int)


# 3. Fill2
fill2_cols = [col for col in tst_cols if "_Fill2" in col]

# 1) 컬럼 중 전체가 결측치인 컬럼 제외
fill2_main_cols = [col for col in fill2_cols if test[col].nunique() >= 1]

tst_fill2 = test[fill2_main_cols]

# 2) 값이 1개인 컬럼 drop
drop_tst_fill2 = tst_fill2.drop(columns=[
    "Wip Line_Fill2", 
    "Process Desc._Fill2",
    "Insp. Seq No._Fill2",
    "Insp Judge Code_Fill2",
    "CURE END POSITION Θ Collect Result_Fill2",
    "CURE STANDBY POSITION X Collect Result_Fill2",
    "CURE STANDBY POSITION Θ Collect Result_Fill2",
    "CURE START POSITION Θ Collect Result_Fill2",
    "DISCHARGED SPEED OF RESIN Collect Result_Fill2",
    "DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill2",
    "DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill2",
    "DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill2",
    "Dispense Volume(Stage1) Collect Result_Fill2",
    "Dispense Volume(Stage2) Collect Result_Fill2",
    "Dispense Volume(Stage3) Collect Result_Fill2"
])

# 3) 결측치, 이상치 대체, 값 변환
# drop_tst_fill2["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2"] = drop_tst_fill2["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2"].replace("OK", np.nan).astype(float)
# drop_tst_fill2["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2"].fillna(drop_tst_fill2["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2"].mean(), inplace=True)
# drop_tst_fill2["HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2"] = drop_tst_fill2["HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2"].fillna("NG").replace({"OK": 0, "NG": 1}).astype(int)


# 4. AutoClave
autoclave_cols = [col for col in tst_cols if "_AutoClave" in col]

# 1) 컬럼 중 전체가 결측치인 컬럼 제외
autoclave_main_cols = [col for col in autoclave_cols if test[col].nunique() >= 1]

tst_autoclave = test[autoclave_main_cols]

# 2) 값이 1개인 컬럼 drop
drop_tst_autoclave = tst_autoclave.drop(columns=[
    "Wip Line_AutoClave", 
    "Process Desc._AutoClave",
    "Equipment_AutoClave",
    "Insp. Seq No._AutoClave",
    "Insp Judge Code_AutoClave",
    "1st Pressure Judge Value_AutoClave",
    "2nd Pressure Judge Value_AutoClave",
    "3rd Pressure Judge Value_AutoClave",
])

# 3) 결측치, 이상치 대체, 값 변환
# drop_tst_autoclave["GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave"] = drop_tst_autoclave["GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave"].fillna("NG").replace({"OK": 0, "NG": 1}).astype(int)
# drop_tst_autoclave["GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave"] = drop_tst_autoclave["GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave"].fillna("NG").replace({"OK": 0, "NG": 1}).astype(int)
# drop_tst_autoclave["Chamber Temp. Judge Value_AutoClave"] = drop_tst_autoclave["Chamber Temp. Judge Value_AutoClave"].replace({"OK": 0, "NG": 1}).astype(int)


# 5. 데이터 병합
print(test.shape)
test_concat = pd.concat([test["Set ID"],
                         drop_tst_dam, 
                         drop_tst_fill1, 
                         drop_tst_fill2, 
                         drop_tst_autoclave,
                         test["target"].fillna("Normal")], axis=1)

# 값 변환
test_concat["target"] = test_concat["target"].replace({"Normal": 0, "AbNormal": 1}).astype(int)

print(drop_tst_dam.shape[1],
      drop_tst_fill1.shape[1],
      drop_tst_fill2.shape[1],
      drop_tst_autoclave.shape[1], 
      test_concat.shape)

(17361, 469)
71 35 34 14 (17361, 156)


In [6]:
train = train_concat.copy()
test = test_concat.copy()

## 밀려있는 값들 정렬

In [7]:
def shift_columns(df, suffix):
    shift_col = list(df.loc[:, f'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_{suffix}':f'WorkMode Collect Result_{suffix}'].columns)
    mask = (df[f'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_{suffix}'].isna()) | \
           (df[f'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_{suffix}'] == 'OK')
    df.loc[mask, shift_col] = df.loc[mask, shift_col].shift(-1, axis=1)
    return df

train = shift_columns(train, 'Dam')
train = shift_columns(train, 'Fill1')
train = shift_columns(train, 'Fill2')

test = shift_columns(test, 'Dam')
test = shift_columns(test, 'Fill1')
test = shift_columns(test, 'Fill2')

/tmp/ipykernel_5885/1863811897.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.     0.012  0.    ...  0.    -0.019  0.   ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[mask, shift_col] = df.loc[mask, shift_col].shift(-1, axis=1)
/tmp/ipykernel_5885/1863811897.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[114.612 114.612  85.    ...  85.    114.612  85.   ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[mask, shift_col] = df.loc[mask, shift_col].shift(-1, axis=1)
/tmp/ipykernel_5885/1863811897.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.054  0.     0.    ...  0.     0.     0.   ]' has dtype incompatible with int64, please explicitly cast to a compa

In [8]:
train["WorkMode Collect Result_Dam"] = train["WorkMode Collect Result_Dam"].fillna(-1)
train["WorkMode Collect Result_Fill1"] = train["WorkMode Collect Result_Fill1"].fillna(-1)
train["WorkMode Collect Result_Fill2"] = train["WorkMode Collect Result_Fill2"].fillna(-1)

In [9]:
# NaN 값 drop
train = train.dropna(axis=1, how='any')
train

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,Dispense Volume(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam,HEAD Standby Position X Collect Result_Dam,HEAD Standby Position Y Collect Result_Dam,HEAD Standby Position Z Collect Result_Dam,Head Clean Position X Collect Result_Dam,Head Clean Position Y Collect Result_Dam,Head Clean Position Z Collect Result_Dam,Head Purge Position X Collect Result_Dam,Head Purge Position Y Collect Result_Dam,Head Purge Position Z Collect Result_Dam,Head Zero Position X Collect Result_Dam,Head Zero Position Y Collect Result_Dam,Head Zero Position Z Collect Result_Dam,Machine Tact time Collect Result_Dam,PalletID Collect Result_Dam,Production Qty Collect Result_Dam,Receip No Collect Result_Dam,Stage1 Circle1 Distance Speed Collect Result_Dam,Stage1 Circle2 Distance Speed Collect Result_Dam,Stage1 Circle3 Distance Speed Collect Result_Dam,Stage1 Circle4 Distance Speed Collect Result_Dam,Stage1 Line1 Distance Speed Collect Result_Dam,Stage1 Line2 Distance Speed Collect Result_Dam,Stage1 Line3 Distance Speed Collect Result_Dam,Stage1 Line4 Distance Speed Collect Result_Dam,Stage2 Circle1 Distance Speed Collect Result_Dam,Stage2 Circle2 Distance Speed Collect Result_Dam,Stage2 Circle3 Distance Speed Collect Result_Dam,Stage2 Circle4 Distance Speed Collect Result_Dam,Stage2 Line1 Distance Speed Collect Result_Dam,Stage2 Line2 Distance Speed Collect Result_Dam,Stage2 Line3 Distance Speed Collect Result_Dam,Stage2 Line4 Distance Speed Collect Result_Dam,Stage3 Circle1 Distance Speed Collect Result_Dam,Stage3 Circle2 Distance Speed Collect Result_Dam,Stage3 Circle3 Distance Speed Collect Result_Dam,Stage3 Circle4 Distance Speed Collect Result_Dam,Stage3 Line1 Distance Speed Collect Result_Dam,Stage3 Line2 Distance Speed Collect Result_Dam,Stage3 Line3 Distance Speed Collect Result_Dam,Stage3 Line4 Distance Speed Collect Result_Dam,THICKNESS 1 Collect Result_Dam,THICKNESS 2 Collect Result_Dam,THICKNESS 3 Collect Result_Dam,WorkMode Collect Result_Dam,Equipment_Fill1,Model.Suffix_Fill1,Workorder_Fill1,Collect Date_Fill1,DISCHARGED SPEED OF RESIN Collect Result_Fill1,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1,Dispense Volume(Stage1) Collect Result_Fill1,Dispense Volume(Stage2) Collect Result_Fill1,Dispense Volume(Stage3) Collect Result_Fill1,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1,HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1,HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1,HEAD Standby Position X Collect Result_Fill1,HEAD Standby Position Y Collect Result_Fill1,HEAD Standby Position Z Collect Result_Fill1,Head 

## 특징 추출

In [10]:
train['Collect Date_Dam'] = pd.to_datetime(train['Collect Date_Dam'])
train['Collect Date_Fill1'] = pd.to_datetime(train['Collect Date_Fill1'])
train['Collect Date_Fill2'] = pd.to_datetime(train['Collect Date_Fill2'])
train['Collect Date_AutoClave'] = pd.to_datetime(train['Collect Date_AutoClave'])

# 각 두 날짜 사이의 시간 차이를 계산
train['Time_diff_1_2'] = (train['Collect Date_Fill1'] - train['Collect Date_Dam']).dt.total_seconds() / 3600  # 시간 단위로 변환
train['Time_diff_2_3'] = (train['Collect Date_Fill2'] - train['Collect Date_Fill1']).dt.total_seconds() / 3600
train['Time_diff_3_4'] = (train['Collect Date_AutoClave'] - train['Collect Date_Fill2']).dt.total_seconds() / 3600

train[['Time_diff_1_2', 'Time_diff_2_3', 'Time_diff_3_4']]

,Time_diff_1_2,Time_diff_2_3,Time_diff_3_4
0,0.166667,0.0,0.500000
1,0.000000,0.0,0.500000
2,0.000000,0.0,0.666667
3,0.000000,0.0,0.666667
4,0.000000,0.0,0.666667
...,...,...,...
40501,0.000000,0.0,0.666667
40502,0.000000,0.0,0.666667
40503,0.166667,0.0,0.500000
40504,0.000000,0.0,0.666667


In [11]:
test['Collect Date_Dam'] = pd.to_datetime(test['Collect Date_Dam'])
test['Collect Date_Fill1'] = pd.to_datetime(test['Collect Date_Fill1'])
test['Collect Date_Fill2'] = pd.to_datetime(test['Collect Date_Fill2'])
test['Collect Date_AutoClave'] = pd.to_datetime(test['Collect Date_AutoClave'])

# 각 두 날짜 사이의 시간 차이를 계산
test['Time_diff_1_2'] = (test['Collect Date_Fill1'] - test['Collect Date_Dam']).dt.total_seconds() / 3600  # 시간 단위로 변환
test['Time_diff_2_3'] = (test['Collect Date_Fill2'] - test['Collect Date_Fill1']).dt.total_seconds() / 3600
test['Time_diff_3_4'] = (test['Collect Date_AutoClave'] - test['Collect Date_Fill2']).dt.total_seconds() / 3600

test[['Time_diff_1_2', 'Time_diff_2_3', 'Time_diff_3_4']]

,Time_diff_1_2,Time_diff_2_3,Time_diff_3_4
0,0.166667,0.000000,0.500000
1,0.000000,0.000000,0.666667
2,0.166667,0.000000,0.500000
3,0.000000,0.000000,0.500000
4,0.000000,0.166667,0.500000
...,...,...,...
17356,0.000000,0.000000,0.666667
17357,0.000000,0.000000,0.500000
17358,0.000000,0.000000,0.500000
17359,0.000000,0.000000,0.500000


In [12]:
df_hand = pd.read_excel(os.path.join(ROOT_DIR, "hand_data.xlsx"))
df_hand['investigation_time'] = df_hand['Fill cure energy (mJ) - Every break time'] / df_hand['Fill cure power (mW) - Every break time']
df_hand

,day,start_time,end_time,Dam Thickness - 1time / day,Fill thickness - 1time / day,Fill cure energy (mJ) - Every break time,Fill cure power (mW) - Every break time,Line Temp. - Every break time,Line humidity - Every break time,OCR gap - Line#1 stage1,OCR gap - Line#1 stage2,OCR gap - Line#1 stage3,OCR gap - Line #2 stage1,OCR gap - Line #2 stage2,OCR gap - Line #2 stage3,investigation_time
0,240319,08:00:00,10:30:00,454.0,193.0,1728.0,2919.0,23.4,56.6,1.2,1.000,1.200,1.2,0.900,1.4,0.591984
1,240319,10:45:00,12:00:00,454.0,193.0,1754.0,2962.0,24.0,50.9,1.2,1.000,1.200,1.2,0.900,1.4,0.592167
2,240319,13:00:00,15:00:00,454.0,193.0,1708.0,2887.0,24.5,49.6,1.2,1.000,1.200,1.2,0.900,1.4,0.591618
3,240319,15:15:00,18:00:00,454.0,193.0,1751.0,2946.0,24.7,53.4,1.2,1.000,1.200,1.2,1.200,1.4,0.594365
4,240319,18:30:00,20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,240428,08:00:00,10:30:00,478.0,232.0,1642.0,2952.0,24.6,53.0,1.0,1.232,1.466,1.2,1.167,1.5,0.556233
101,240428,10:45:00,12:00:00,478.0,232.0,1679.0,2942.0,24.8,52.7,1.0,1.232,1.466,1.2,1.167,1.5,0.570700
102,240428,13:00:00,15:00:00,478.0,232.0,1552.0,2941.0,24.6,55.5,1.0,1.232,1.466,1.2,1.167,1.5,0.527712
103,240428,15:15:00,18:00:00,478.0,232.0,1552.0,2934.0,24.3,54.6,1.0,1.232,1.466,1.2,1.167,1.5,0.528971


In [13]:
from datetime import timedelta

train['Collect Date_Dam'] = pd.to_datetime(train['Collect Date_Dam'], format='%y%m%d')
test['Collect Date_Dam'] = pd.to_datetime(test['Collect Date_Dam'], format='%y%m%d')

df_hand['day'] = pd.to_datetime(df_hand['day'], format='%y%m%d')

df_hand['start_time'] = pd.to_timedelta(df_hand['start_time'].astype(str))
df_hand['end_time'] = pd.to_timedelta(df_hand['end_time'].astype(str))

df_hand['start_datetime'] = df_hand['day'] + df_hand['start_time']
df_hand['end_datetime'] = df_hand['day'] + df_hand['end_time']

In [14]:
from datetime import datetime

def merge_by_datetime(row):
    cutoff_date = datetime(2024, 3, 19, 8, 0, 0)
    
    collect_date = pd.to_datetime(row['Collect Date_Dam'])
    
    if collect_date <= cutoff_date:
        mask = (df_hand['start_datetime'] <= collect_date) & (collect_date <= df_hand['end_datetime'])
        if mask.any():
            return df_hand[mask].iloc[0]
    else:
        if not df_hand.empty:
            start_diff = abs(df_hand['start_datetime'] - collect_date)
            end_diff = abs(df_hand['end_datetime'] - collect_date)
            
            if start_diff.min() <= end_diff.min():
                return df_hand.loc[start_diff.idxmin()]
            else:
                return df_hand.loc[end_diff.idxmin()]
    
    return pd.Series([np.nan] * len(df_hand.columns), index=df_hand.columns)

df_hand['start_datetime'] = pd.to_datetime(df_hand['start_datetime'])
df_hand['end_datetime'] = pd.to_datetime(df_hand['end_datetime'])

merged_data = train.apply(merge_by_datetime, axis=1)
train = pd.concat([train, merged_data], axis=1)

merged_data = test.apply(merge_by_datetime, axis=1)
test = pd.concat([test, merged_data], axis=1)

train

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,Dispense Volume(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam,HEAD Standby Position X Collect Result_Dam,HEAD Standby Position Y Collect Result_Dam,HEAD Standby Position Z Collect Result_Dam,Head Clean Position X Collect Result_Dam,Head Clean Position Y Collect Result_Dam,Head Clean Position Z Collect Result_Dam,Head Purge Position X Collect Result_Dam,Head Purge Position Y Collect Result_Dam,Head Purge Position Z Collect Result_Dam,Head Zero Position X Collect Result_Dam,Head Zero Position Y Collect Result_Dam,Head Zero Position Z Collect Result_Dam,Machine Tact time Collect Result_Dam,PalletID Collect Result_Dam,Production Qty Collect Result_Dam,Receip No Collect Result_Dam,Stage1 Circle1 Distance Speed Collect Result_Dam,Stage1 Circle2 Distance Speed Collect Result_Dam,Stage1 Circle3 Distance Speed Collect Result_Dam,Stage1 Circle4 Distance Speed Collect Result_Dam,Stage1 Line1 Distance Speed Collect Result_Dam,Stage1 Line2 Distance Speed Collect Result_Dam,Stage1 Line3 Distance Speed Collect Result_Dam,Stage1 Line4 Distance Speed Collect Result_Dam,Stage2 Circle1 Distance Speed Collect Result_Dam,Stage2 Circle2 Distance Speed Collect Result_Dam,Stage2 Circle3 Distance Speed Collect Result_Dam,Stage2 Circle4 Distance Speed Collect Result_Dam,Stage2 Line1 Distance Speed Collect Result_Dam,Stage2 Line2 Distance Speed Collect Result_Dam,Stage2 Line3 Distance Speed Collect Result_Dam,Stage2 Line4 Distance Speed Collect Result_Dam,Stage3 Circle1 Distance Speed Collect Result_Dam,Stage3 Circle2 Distance Speed Collect Result_Dam,Stage3 Circle3 Distance Speed Collect Result_Dam,Stage3 Circle4 Distance Speed Collect Result_Dam,Stage3 Line1 Distance Speed Collect Result_Dam,Stage3 Line2 Distance Speed Collect Result_Dam,Stage3 Line3 Distance Speed Collect Result_Dam,Stage3 Line4 Distance Speed Collect Result_Dam,THICKNESS 1 Collect Result_Dam,THICKNESS 2 Collect Result_Dam,THICKNESS 3 Collect Result_Dam,WorkMode Collect Result_Dam,Equipment_Fill1,Model.Suffix_Fill1,Workorder_Fill1,Collect Date_Fill1,DISCHARGED SPEED OF RESIN Collect Result_Fill1,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1,Dispense Volume(Stage1) Collect Result_Fill1,Dispense Volume(Stage2) Collect Result_Fill1,Dispense Volume(Stage3) Collect Result_Fill1,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1,HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1,HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1,HEAD Standby Position X Collect Result_Fill1,HEAD Standby Position Y Collect Result_Fill1,HEAD Standby Position Z Collect Result_Fill1,Head 

In [15]:
train.drop(columns=['day', 'start_time', 'end_time', 'start_datetime', 'end_datetime'], inplace=True)
test.drop(columns=['day', 'start_time', 'end_time', 'start_datetime', 'end_datetime'], inplace=True)

In [16]:
# Workorder_Dam별 같은 특징을 포착하기 위해 slice

# train['Workorder_Dam_slice_1'] = train['Workorder_Dam'].str.slice(0, 1)
train['Workorder_Dam_slice_2'] = train['Workorder_Dam'].str.slice(0, 2)
# train['Workorder_Dam_slice_3'] = train['Workorder_Dam'].str.slice(0, 3)
# train['Workorder_Dam_slice_4'] = train['Workorder_Dam'].str.slice(0, 4)
train['Workorder_Dam_slice_6'] = train['Workorder_Dam'].str.slice(0, 6)

# test['Workorder_Dam_slice_1'] = test['Workorder_Dam'].str.slice(0, 1)
test['Workorder_Dam_slice_2'] = test['Workorder_Dam'].str.slice(0, 2)
# test['Workorder_Dam_slice_3'] = test['Workorder_Dam'].str.slice(0, 3)
# test['Workorder_Dam_slice_4'] = test['Workorder_Dam'].str.slice(0, 4)
test['Workorder_Dam_slice_6'] = test['Workorder_Dam'].str.slice(0, 6)


In [17]:
train['1st Pressure AutoClave'] = train['1st Pressure 1st Pressure Unit Time_AutoClave'] / train['1st Pressure Collect Result_AutoClave']
train['2nd Pressure AutoClave'] = train['2nd Pressure Unit Time_AutoClave'] / train['2nd Pressure Collect Result_AutoClave']
train['3rd Pressure AutoClave'] = train['3rd Pressure Unit Time_AutoClave'] / train['3rd Pressure Collect Result_AutoClave']
train['Chamber Temp Time'] = train['Chamber Temp. Unit Time_AutoClave'] / train['Chamber Temp. Collect Result_AutoClave']

test['1st Pressure AutoClave'] = test['1st Pressure 1st Pressure Unit Time_AutoClave'] / test['1st Pressure Collect Result_AutoClave']
test['2nd Pressure AutoClave'] = test['2nd Pressure Unit Time_AutoClave'] / test['2nd Pressure Collect Result_AutoClave']
test['3rd Pressure AutoClave'] = test['3rd Pressure Unit Time_AutoClave'] / test['3rd Pressure Collect Result_AutoClave']
test['Chamber Temp Time'] = test['Chamber Temp. Unit Time_AutoClave'] / test['Chamber Temp. Collect Result_AutoClave']


In [18]:
train['Stage1 Circle value'] = train['Stage1 Circle1 Distance Speed Collect Result_Dam'] + train['Stage1 Circle2 Distance Speed Collect Result_Dam'] + train['Stage1 Circle3 Distance Speed Collect Result_Dam'] + train['Stage1 Circle4 Distance Speed Collect Result_Dam']
train['Stage2 Circle value'] = train['Stage2 Circle1 Distance Speed Collect Result_Dam'] + train['Stage2 Circle2 Distance Speed Collect Result_Dam'] + train['Stage2 Circle3 Distance Speed Collect Result_Dam'] + train['Stage2 Circle4 Distance Speed Collect Result_Dam']
train['Stage3 Circle value'] = train['Stage3 Circle1 Distance Speed Collect Result_Dam'] + train['Stage3 Circle2 Distance Speed Collect Result_Dam'] + train['Stage3 Circle3 Distance Speed Collect Result_Dam'] + train['Stage3 Circle4 Distance Speed Collect Result_Dam']
train['Stage1 Line value'] = train['Stage1 Line1 Distance Speed Collect Result_Dam'] + train['Stage1 Line2 Distance Speed Collect Result_Dam'] + train['Stage1 Line3 Distance Speed Collect Result_Dam'] + train['Stage1 Line4 Distance Speed Collect Result_Dam']
train['Stage2 Line value'] = train['Stage2 Line1 Distance Speed Collect Result_Dam'] + train['Stage2 Line2 Distance Speed Collect Result_Dam'] + train['Stage2 Line3 Distance Speed Collect Result_Dam'] + train['Stage2 Line4 Distance Speed Collect Result_Dam']
train['Stage3 Line value'] = train['Stage3 Line1 Distance Speed Collect Result_Dam'] + train['Stage3 Line2 Distance Speed Collect Result_Dam'] + train['Stage3 Line3 Distance Speed Collect Result_Dam'] + train['Stage3 Line4 Distance Speed Collect Result_Dam']
train['THICKNESS value'] = train['THICKNESS 1 Collect Result_Dam'] * train['THICKNESS 2 Collect Result_Dam'] * train['Stage3 Line3 Distance Speed Collect Result_Dam'] * train['THICKNESS 3 Collect Result_Dam']

test['Stage1 Circle value'] = test['Stage1 Circle1 Distance Speed Collect Result_Dam'] + test['Stage1 Circle2 Distance Speed Collect Result_Dam'] + test['Stage1 Circle3 Distance Speed Collect Result_Dam'] + test['Stage1 Circle4 Distance Speed Collect Result_Dam']
test['Stage2 Circle value'] = test['Stage2 Circle1 Distance Speed Collect Result_Dam'] + test['Stage2 Circle2 Distance Speed Collect Result_Dam'] + test['Stage2 Circle3 Distance Speed Collect Result_Dam'] + test['Stage2 Circle4 Distance Speed Collect Result_Dam']
test['Stage3 Circle value'] = test['Stage3 Circle1 Distance Speed Collect Result_Dam'] + test['Stage3 Circle2 Distance Speed Collect Result_Dam'] + test['Stage3 Circle3 Distance Speed Collect Result_Dam'] + test['Stage3 Circle4 Distance Speed Collect Result_Dam']
test['Stage1 Line value'] = test['Stage1 Line1 Distance Speed Collect Result_Dam'] + test['Stage1 Line2 Distance Speed Collect Result_Dam'] + test['Stage1 Line3 Distance Speed Collect Result_Dam'] + test['Stage1 Line4 Distance Speed Collect Result_Dam']
test['Stage2 Line value'] = test['Stage2 Line1 Distance Speed Collect Result_Dam'] + test['Stage2 Line2 Distance Speed Collect Result_Dam'] + test['Stage2 Line3 Distance Speed Collect Result_Dam'] + test['Stage2 Line4 Distance Speed Collect Result_Dam']
test['Stage3 Line value'] = test['Stage3 Line1 Distance Speed Collect Result_Dam'] + test['Stage3 Line2 Distance Speed Collect Result_Dam'] + test['Stage3 Line3 Distance Speed Collect Result_Dam'] + test['Stage3 Line4 Distance Speed Collect Result_Dam']
test['THICKNESS value'] = test['THICKNESS 1 Collect Result_Dam'] * test['THICKNESS 2 Collect Result_Dam'] * test['Stage3 Line3 Distance Speed Collect Result_Dam'] * test['THICKNESS 3 Collect Result_Dam']


In [19]:
train['DIS_per_TIME1'] = train['DISCHARGED SPEED OF RESIN Collect Result_Dam'] / (train['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam'] / train['Dispense Volume(Stage1) Collect Result_Dam'])
train['DIS_per_TIME2'] = train['DISCHARGED SPEED OF RESIN Collect Result_Dam'] / (train['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam'] / train['Dispense Volume(Stage2) Collect Result_Dam'])
train['DIS_per_TIME3'] = train['DISCHARGED SPEED OF RESIN Collect Result_Dam'] / (train['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam'] / train['Dispense Volume(Stage3) Collect Result_Dam'])

test['DIS_per_TIME1'] = test['DISCHARGED SPEED OF RESIN Collect Result_Dam'] / (test['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam'] / test['Dispense Volume(Stage1) Collect Result_Dam'])
test['DIS_per_TIME2'] = test['DISCHARGED SPEED OF RESIN Collect Result_Dam'] / (test['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam'] / test['Dispense Volume(Stage2) Collect Result_Dam'])
test['DIS_per_TIME3'] = test['DISCHARGED SPEED OF RESIN Collect Result_Dam'] / (test['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam'] / test['Dispense Volume(Stage3) Collect Result_Dam'])

In [20]:
train['process_time'] = train['Machine Tact time Collect Result_Dam'] + train['Machine Tact time Collect Result_Fill1'] + train['Machine Tact time Collect Result_Fill2']
test['process_time'] = test['Machine Tact time Collect Result_Dam'] + test['Machine Tact time Collect Result_Fill1'] + test['Machine Tact time Collect Result_Fill2']

In [21]:
single_unique_cols = []

for column in train.columns:
    if len(train[column].unique()) == 1:
        single_unique_cols.append(column)

train.drop(single_unique_cols, axis=1, inplace=True)
len(single_unique_cols)

24

In [22]:
# 숫자형으로 모델에 넣어봤지만 성능 저하로 인하여 모든 컬럼을 category로 사용함

num_col = [
#     'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
#     'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
#     'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam',
    # 'Dispense Volume(Stage1) Collect Result_Dam',
    # 'Dispense Volume(Stage2) Collect Result_Dam',
    # 'Dispense Volume(Stage3) Collect Result_Dam',
    # 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
    # 'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam',
    # 'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam',
    # 'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam',
    # # 'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam',
    # # 'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam',
    # # 'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam',
    # # 'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam',
    # 'Machine Tact time Collect Result_Dam',
    # 'Production Qty Collect Result_Dam',
    # 'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1',
    # 'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1',
    # 'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1',
    # 'Dispense Volume(Stage1) Collect Result_Fill1',
    # 'Dispense Volume(Stage2) Collect Result_Fill1',
    # 'Dispense Volume(Stage3) Collect Result_Fill1',
    # # 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1',
    # # 'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1',
    # # 'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1',
    # # 'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1',
    # # 'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1',
    # # 'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1',
    # # 'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1',
    # # 'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1',
    # 'Machine Tact time Collect Result_Fill1',
    # 'Production Qty Collect Result_Fill1',
    # 'CURE SPEED Collect Result_Fill2',
    # 'Machine Tact time Collect Result_Fill2',
    # 'Production Qty Collect Result_Fill2',
#     '1st Pressure Collect Result_AutoClave',
#     '2nd Pressure Collect Result_AutoClave',
#     '3rd Pressure Collect Result_AutoClave',
#     'Chamber Temp. Collect Result_AutoClave',
#     'Chamber Temp. Unit Time_AutoClave',
#     '1st Pressure AutoClave',
#     '2st Pressure AutoClave',
#     '3st Pressure AutoClave',
#     'Chamber Temp Time',
#     'DIS_per_TIME1',
#     'DIS_per_TIME2',
#     'DIS_per_TIME3',
#     'process_time'
]

In [23]:
selected_columns = [col for col in train.columns if col not in num_col]

for col in selected_columns:
    train[col] = train[col].astype(str)
    test[col] = test[col].astype(str)

selected_columns.remove('target')

In [24]:
train.drop(['Production Qty Collect Result_Fill1', 'Production Qty Collect Result_Fill2', 'Workorder_Fill1', 'Workorder_Fill2', 'Workorder_AutoClave', 'PalletID Collect Result_Fill1', 'PalletID Collect Result_Fill2'], axis=1, inplace=True)

In [25]:
selected_columns.remove('Workorder_Fill1')
selected_columns.remove('Workorder_Fill2')
selected_columns.remove('PalletID Collect Result_Fill1')
selected_columns.remove('PalletID Collect Result_Fill2')
selected_columns.remove('Workorder_AutoClave')
selected_columns.remove('Production Qty Collect Result_Fill1')
selected_columns.remove('Production Qty Collect Result_Fill2')

### 데이터 분할

In [26]:
# 최대한 밸런스 있게 평가하기 위해 2가지 stratify 선택

from sklearn.model_selection import train_test_split

stratify_columns = train[["target", "Workorder_Dam_slice_2"]]

df_train, df_val = train_test_split(
    train,
    test_size=0.2,
    stratify=stratify_columns,
    random_state=42,
)

In [27]:
train_x = df_train.drop(columns=['target'])
train_y = df_train['target']

train_y = train_y.replace({'Normal': 0, 'AbNormal': 1})

In [28]:
valid_x = df_val[train_x.columns]
valid_y = df_val['target']

valid_y = valid_y.replace({'Normal': 0, 'AbNormal': 1})

## 3. 모델 학습

### optuna를 통한 hyperparameter 찾기

In [29]:
# !pip install optuna

In [30]:
# from catboost import CatBoostClassifier
# from sklearn.metrics import f1_score
# import optuna
# from optuna.samplers import TPESampler
# from optuna.pruners import MedianPruner

# def objective(trial):
#     params = {
#         'objective': 'Logloss',
#         'eval_metric': 'F1',
#         'random_seed': 42,
#         'learning_rate': 0.027481,
#         'cat_features': selected_columns,
#         'grow_policy': 'SymmetricTree',

#         'depth': trial.suggest_int('depth', 3, 8),
#         'iterations': trial.suggest_int('iterations', 100, 1500),
#         'border_count': trial.suggest_int('border_count', 8, 500),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 16),
#         'max_ctr_complexity': trial.suggest_int('max_ctr_complexity', 1, 15),

#         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 16),
#         'scale_pos_weight': trial.suggest_float('scale_pos_weight', 3, 9),
#         'random_strength': trial.suggest_float('random_strength', 0, 1),
#         'rsm': trial.suggest_float('rsm', 0, 1),
#         'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),


#         'boosting_type': trial.suggest_categorical('boosting_type', ['Plain', 'Ordered']),
#     }
    
#     model = CatBoostClassifier(**params)

#     early_stopping_rounds = 100

#     model.fit(
#         train_x, train_y,
#         eval_set=(valid_x, valid_y),
#         verbose=False,
#         early_stopping_rounds=early_stopping_rounds,
#         use_best_model=True
#     )
    
#     preds = model.predict(valid_x)
    
#     score = f1_score(valid_y, preds, average='binary')
    
#     trial.report(score, step=model.tree_count_)
    
#     if trial.should_prune():
#         raise optuna.TrialPruned()
    
#     return score

# train_y = train_y.astype(int)
# valid_y = valid_y.astype(int)

# study = optuna.create_study(
#     direction='maximize',
#     sampler=TPESampler(seed=42),
#     pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=50, interval_steps=20)
# )

# study.optimize(objective, n_trials=1000)

# print("Best trial:")
# trial = study.best_trial
# print(f"  Value: {trial.value:.5f}")
# print("  Params: ")
# for key, value in trial.params.items():
#     print(f"    {key}: {value}")

### 모델 학습

In [31]:
# !pip install catboost

In [32]:
from catboost import CatBoostClassifier
model_params= {'depth': 4, 
               'iterations': 1137, 
               'border_count': 250, 
               'min_data_in_leaf': 13, 
               'max_ctr_complexity': 10, 
               'l2_leaf_reg': 8.222740592143833, 
               'scale_pos_weight': 7.4505089793308645, 
               'random_strength': 0.930060373829028, 
               'rsm': 0.6077126760391974, 
               'bagging_temperature': 0.06707949399380098, 
               'boosting_type': 'Plain',
               
                'objective': 'Logloss',
                'eval_metric': 'F1',
                'random_seed': 42,
                'learning_rate': 0.027481,
                'cat_features': selected_columns,
                'grow_policy': 'SymmetricTree',
                'one_hot_max_size': 5,
                'early_stopping_rounds': 300,
                'use_best_model': True,
              
              }

model_cat = CatBoostClassifier(**model_params)

model_cat.fit(
    train_x, train_y, 
    eval_set=(valid_x, valid_y), 
    verbose=100
    )

0:	learn: 0.1479575	test: 0.1422340	best: 0.1422340 (0)	total: 362ms	remaining: 6m 50s
100:	learn: 0.2862593	test: 0.3307402	best: 0.3307402 (98)	total: 28.4s	remaining: 4m 51s
200:	learn: 0.3112844	test: 0.3633620	best: 0.3634441 (194)	total: 55.2s	remaining: 4m 17s
300:	learn: 0.3346652	test: 0.3766849	best: 0.3766849 (300)	total: 1m 21s	remaining: 3m 47s
400:	learn: 0.3518398	test: 0.4033754	best: 0.4034628 (397)	total: 1m 48s	remaining: 3m 19s
500:	learn: 0.3641130	test: 0.4110490	best: 0.4114009 (462)	total: 2m 19s	remaining: 2m 56s
600:	learn: 0.3837912	test: 0.4122563	best: 0.4151226 (579)	total: 2m 49s	remaining: 2m 31s
700:	learn: 0.4026166	test: 0.4158638	best: 0.4167449 (656)	total: 3m 17s	remaining: 2m 3s
800:	learn: 0.4057445	test: 0.4177380	best: 0.4187973 (752)	total: 3m 47s	remaining: 1m 35s
900:	learn: 0.4185887	test: 0.4145491	best: 0.4187973 (752)	total: 4m 17s	remaining: 1m 7s
1000:	learn: 0.4258701	test: 0.4172104	best: 0.4199538 (986)	total: 4m 47s	remaining: 39.1

In [33]:
y_pred_proba = model_cat.predict_proba(valid_x)[:, 1]
valid_y = valid_y.astype(int)

thresholds = np.arange(0.1, 1, 0.01)
f1_scores = [f1_score(valid_y, y_pred_proba >= threshold) for threshold in thresholds]

best_threshold = thresholds[np.argmax(f1_scores)]
best_f1 = max(f1_scores)

print(f"최적 임계값: {best_threshold:.2f}")
print(f"최적 F1 점수: {best_f1:.4f}")

y_pred_optimal = (y_pred_proba >= best_threshold).astype(int)
f1_optimal = f1_score(valid_y, y_pred_optimal)
print(f"최적 임계값 적용 후 F1 점수: {f1_optimal:.4f}")

최적 임계값: 0.54
최적 F1 점수: 0.3222
최적 임계값 적용 후 F1 점수: 0.3222


In [34]:
p = model_cat.predict_proba(valid_x)

threshold = 0.5
p = np.where(p[:, 1] >= threshold, 1, 0)

p = pd.Series(p)

valid_y = valid_y.astype(int)
f1 = f1_score(valid_y, p)
pre = precision_score(valid_y, p)
re = recall_score(valid_y, p)
np.round(f1)
print(np.round(f1, 4), np.round(pre, 4), np.round(re, 4))
print(p.value_counts())

0.3098 0.327 0.2942
0    7680
1     422
Name: count, dtype: int64


In [35]:
importance = model_cat.feature_importances_

importance_df = pd.DataFrame({
    'Feature': train_x.columns,
    'Importance': importance
})

importance_df = importance_df.sort_values(by='Importance', ascending=False)

importance_df[:20]

,Feature,Importance
2,Workorder_Dam,6.667371
117,Time_diff_3_4,4.698200
105,Collect Date_AutoClave,4.552709
132,Workorder_Dam_slice_6,4.492686
30,PalletID Collect Result_Dam,4.143544
31,Production Qty Collect Result_Dam,3.010256
115,Time_diff_1_2,2.912443
101,Machine Tact time Collect Result_Fill2,2.702022
147,process_time,2.665760
85,Collect Date_Fill2,2.626322


In [36]:
try:
    del model_params['use_best_model']
except:
    pass

best_iter = model_cat.best_iteration_
model_params['iterations'] = int(best_iter * 1.2)
model_params['iterations']

1340

In [37]:
model_cat_all = CatBoostClassifier(**model_params)

model_cat_all.fit(
    train.drop(columns=['target']), train['target'], 
    verbose=100
    )

0:	learn: 0.2045083	total: 353ms	remaining: 7m 52s
100:	learn: 0.3055596	total: 33s	remaining: 6m 44s
200:	learn: 0.3278108	total: 1m 5s	remaining: 6m 11s
300:	learn: 0.3431798	total: 1m 38s	remaining: 5m 39s
400:	learn: 0.3560519	total: 2m 9s	remaining: 5m 3s
500:	learn: 0.3698810	total: 2m 44s	remaining: 4m 34s
600:	learn: 0.3852540	total: 3m 19s	remaining: 4m 5s
700:	learn: 0.3992709	total: 3m 55s	remaining: 3m 34s
800:	learn: 0.4103370	total: 4m 31s	remaining: 3m 2s
900:	learn: 0.4229940	total: 5m 4s	remaining: 2m 28s
1000:	learn: 0.4336957	total: 5m 40s	remaining: 1m 55s
1100:	learn: 0.4431215	total: 6m 16s	remaining: 1m 21s
1200:	learn: 0.4476445	total: 6m 52s	remaining: 47.7s
1300:	learn: 0.4531690	total: 7m 28s	remaining: 13.5s
1339:	learn: 0.4547513	total: 7m 43s	remaining: 0us


In [38]:
from sklearn.model_selection import StratifiedKFold

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

fold_results = []

all_predictions = pd.Series(index=train.index)

# 5-fold 교차 검증 수행
for fold, (train_idx, val_idx) in enumerate(skf.split(train, train['target']), 1):
    print(f"Processing fold {fold}")
    
    X_train, y_train = train.drop('target', axis=1).iloc[train_idx], train['target'].iloc[train_idx]
    X_val, y_val = train.drop('target', axis=1).iloc[val_idx], train['target'].iloc[val_idx]
    
    train_proba = model_cat_all.predict_proba(X_train)
    val_proba = model_cat_all.predict_proba(X_val)
    
    threshold = 0.5
    train_preds = np.where(train_proba[:, 1] >= threshold, 1, 0)
    val_preds = np.where(val_proba[:, 1] >= threshold, 1, 0)
    
    all_predictions.iloc[val_idx] = val_preds
    
    y_train = y_train.astype(int)
    y_val = y_val.astype(int)

    train_f1 = f1_score(y_train, train_preds)
    train_pre = precision_score(y_train, train_preds)
    train_re = recall_score(y_train, train_preds)
    
    val_f1 = f1_score(y_val, val_preds)
    val_pre = precision_score(y_val, val_preds)
    val_re = recall_score(y_val, val_preds)
    
    print(f"Fold {fold}:")
    print(f"Train: F1: {train_f1:.4f}, Precision: {train_pre:.4f}, Recall: {train_re:.4f}")
    print(f"Valid: F1: {val_f1:.4f}, Precision: {val_pre:.4f}, Recall: {val_re:.4f}")
    print(f"Predictions distribution - Train: {pd.Series(train_preds).value_counts().to_dict()}, Valid: {pd.Series(val_preds).value_counts().to_dict()}")
    print()
    
    fold_results.append({
        'fold': fold,
        'train_f1': train_f1,
        'train_precision': train_pre,
        'train_recall': train_re,
        'val_f1': val_f1,
        'val_precision': val_pre,
        'val_recall': val_re
    })


train['target'] = train['target'].astype(int)

overall_f1 = f1_score(train['target'], all_predictions)
overall_pre = precision_score(train['target'], all_predictions)
overall_re = recall_score(train['target'], all_predictions)

print("Overall Performance:")
print(f"F1: {overall_f1:.4f}, Precision: {overall_pre:.4f}, Recall: {overall_re:.4f}")
print(f"Predictions distribution: {all_predictions.value_counts().to_dict()}")

results_df = pd.DataFrame(fold_results)
print("\nResults Summary:")
results_df.describe()

Processing fold 1
Fold 1:
Train: F1: 0.4493, Precision: 0.6190, Recall: 0.3527
Valid: F1: 0.4495, Precision: 0.5993, Recall: 0.3596
Predictions distribution - Train: {0: 31333, 1: 1071}, Valid: {0: 7820, 1: 282}

Processing fold 2
Fold 2:
Train: F1: 0.4552, Precision: 0.6202, Recall: 0.3596
Valid: F1: 0.4256, Precision: 0.5932, Recall: 0.3319
Predictions distribution - Train: {0: 31315, 1: 1090}, Valid: {0: 7838, 1: 263}

Processing fold 3
Fold 3:
Train: F1: 0.4438, Precision: 0.6078, Recall: 0.3495
Valid: F1: 0.4717, Precision: 0.6434, Recall: 0.3723
Predictions distribution - Train: {0: 31324, 1: 1081}, Valid: {0: 7829, 1: 272}

Processing fold 4
Fold 4:
Train: F1: 0.4533, Precision: 0.6194, Recall: 0.3574
Valid: F1: 0.4336, Precision: 0.5970, Recall: 0.3404
Predictions distribution - Train: {0: 31320, 1: 1085}, Valid: {0: 7833, 1: 268}

Processing fold 5
Fold 5:
Train: F1: 0.4452, Precision: 0.6083, Recall: 0.3511
Valid: F1: 0.4661, Precision: 0.6418, Recall: 0.3660
Predictions dist

,fold,train_f1,train_precision,train_recall,val_f1,val_precision,val_recall
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,3.000000,0.449362,0.614930,0.354043,0.449309,0.614927,0.354043
std,1.581139,0.004959,0.006313,0.004298,0.019941,0.025351,0.017193
min,1.000000,0.443769,0.607771,0.349468,0.425648,0.593156,0.331915
25%,2.000000,0.445194,0.608295,0.351064,0.433604,0.597015,0.340426
50%,3.000000,0.449339,0.619048,0.352660,0.449468,0.599291,0.359574
75%,4.000000,0.453288,0.619355,0.357447,0.466125,0.641791,0.365957
max,5.000000,0.455219,0.620183,0.359574,0.471698,0.643382,0.372340


In [39]:
importance = model_cat_all.feature_importances_

importance_df = pd.DataFrame({
    'Feature': train_x.columns,
    'Importance': importance
})

importance_df = importance_df.sort_values(by='Importance', ascending=False)

importance_df[:20]

,Feature,Importance
2,Workorder_Dam,8.074701
117,Time_diff_3_4,5.553856
132,Workorder_Dam_slice_6,5.495635
105,Collect Date_AutoClave,5.299882
30,PalletID Collect Result_Dam,3.553413
147,process_time,3.292423
17,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect ...,3.046082
31,Production Qty Collect Result_Dam,2.795030
3,Collect Date_Dam,2.652042
115,Time_diff_1_2,2.480097


## 4. 제출하기

In [40]:
df_test_x = test[list(train_x.columns)]

In [126]:
# 최적의 결과를 찾기위한 threshold 세팅

test_pred = model_cat_all.predict_proba(df_test_x)

threshold = 0.56
pred_labels = np.where(test_pred[:, 1] >= threshold, 1, 0)

test_pred_series = pd.Series(pred_labels)

test_pred_series = test_pred_series.replace({0: 'Normal', 1: 'AbNormal'})

test_ratio = test_pred_series.value_counts()[0] / test_pred_series.value_counts()[1]

test_pred_series.value_counts()

/tmp/ipykernel_5885/1641572712.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  test_ratio = test_pred_series.value_counts()[0] / test_pred_series.value_counts()[1]
/tmp/ipykernel_5885/1641572712.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  test_ratio = test_pred_series.value_counts()[0] / test_pred_series.value_counts()[1]


Normal      16946
AbNormal      415
Name: count, dtype: int64

In [127]:
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = test_pred_series

### 특정 컬럼에 따른 불량 매핑

In [128]:
condition1 = (
    (test['Production Qty Collect Result_Dam'] != test['Production Qty Collect Result_Fill1']) | 
    (test['Production Qty Collect Result_Fill1'] != test['Production Qty Collect Result_Fill2']) | 
    (test['Production Qty Collect Result_Dam'] != test['Production Qty Collect Result_Fill2'])
)

condition2 = (
    (test['Receip No Collect Result_Dam'] != test['Receip No Collect Result_Fill1']) | 
    (test['Receip No Collect Result_Fill1'] != test['Receip No Collect Result_Fill2']) | 
    (test['Receip No Collect Result_Dam'] != test['Receip No Collect Result_Fill2'])
)

combined_condition = condition1 | condition2

test.loc[combined_condition, 'target'] = 'AbNormal'

abnormal_indices = test[combined_condition].index

df_sub.loc[df_sub.index.isin(abnormal_indices), 'target'] = 'AbNormal'

df_sub['target'].value_counts()

target
Normal      16946
AbNormal      415
Name: count, dtype: int64

In [129]:
df_sub.to_csv('submission.csv', index=False)

### 저장 결과 확인

In [130]:
check = pd.read_csv("submission.csv")
check['target'].value_counts()

target
Normal      16946
AbNormal      415
Name: count, dtype: int64